In [7]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow import distribute

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()

K = len(set(y_train))

print("X_train.shape:", X_train.shape)
print("y_train.shape", y_train.shape)

170500096/170498071 [==============================] - 4s 0us/step
X_train.shape: (50000, 32, 32, 3)
y_train.shape (50000,)


In [12]:
# Build the model using the functional API
def create_model():
  i = Input(shape=X_train[0].shape)

  x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
  x = BatchNormalization()(x)
  x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)

  x = Flatten()(x)
  x = Dropout(0.2)(x)
  x = Dense(1024, activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(K, activation='softmax')(x)

  model = Model(i, x)
  return model

In [10]:
strategy = distribute.MirroredStrategy()
# strategy = distribute.experimental.CentralStorageStrategy()
print(f'Number of devices: {strategy.num_replicas_in_sync}')

# Only 1 GPU for this instance

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1


In [13]:
with strategy.scope():
  model = create_model()

  model.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [14]:
r = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=128, epochs=5)
# training process can be speed up with higher batch_size, more GPU or machine instances

Epoch 1/5
391/391 [==============================] - 43s 22ms/step - loss: 1.4775 - accuracy: 0.5112 - val_loss: 3.1886 - val_accuracy: 0.1792
Epoch 2/5
391/391 [==============================] - 7s 18ms/step - loss: 0.8882 - accuracy: 0.6852 - val_loss: 0.8917 - val_accuracy: 0.6858
Epoch 3/5
391/391 [==============================] - 7s 18ms/step - loss: 0.6891 - accuracy: 0.7568 - val_loss: 0.7681 - val_accuracy: 0.7346
Epoch 4/5
391/391 [==============================] - 7s 18ms/step - loss: 0.5696 - accuracy: 0.8008 - val_loss: 0.8562 - val_accuracy: 0.7216
Epoch 5/5
391/391 [==============================] - 7s 18ms/step - loss: 0.4746 - accuracy: 0.8320 - val_loss: 0.7745 - val_accuracy: 0.7445
